Retrieve experiment results

In [11]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [2]
basedirs = [join(get_output_dir(), 'predic    t_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        alpha = config['alpha']
        weights = config['dataset_weights']
        seed = config['seed']
        model = config['model']
        rank = info['rank']
        score = info['score']
        res = {'alpha': alpha, 'seed': seed, 'rank': rank,
              'model': model}
        for dataset, weight in weights.items():
            res['%s_weight' % dataset] = weight
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [12]:
res

,alpha,archi_weight,brainomics_weight,hcp_weight,model,rank,seed,test_archi,test_brainomics,test_hcp,test_mean,train_archi,train_brainomics,train_hcp,train_mean
0,5.179475e-03,1,1,1,trace,21,1230275727,0.844444,0.895981,0.867914,0.866945,0.900000,0.990544,0.883406,0.889430
1,5.179475e-06,1,1,1,logistic,68,1230275727,0.866667,0.882979,0.916346,0.893817,1.000000,1.000000,0.995139,0.995676
2,7.196857e-07,1,1,1,logistic,68,1230275727,0.855556,0.882979,0.910292,0.887555,1.000000,1.000000,0.999938,0.999945
3,2.682696e-04,1,1,1,factored,35,3330543729,0.837607,0.924350,0.938910,0.904774,1.000000,1.000000,0.990156,0.991242
4,2.682696e-04,1,1,1,factored,37,1230275727,0.868376,0.887707,0.922950,0.898513,1.000000,1.000000,0.990033,0.991133
5,1.389495e-02,1,1,1,trace,14,3330543729,0.758974,0.919622,0.794166,0.811114,0.817094,0.957447,0.794746,0.803711
6,1.389495e-05,1,1,1,factored,38,3330543729,0.835043,0.921986,0.913594,0.891469,1.000000,1.000000,1.000000,1.000000
7,2.682696e-07,1,1,1,logistic,68,636201818,0.855556,0.910165,0.913594,0.895121,1.000000,1.000000,1.000000,1.000000
8,5.179475e-03,1,1,1,logistic,68,636201818,0.701709,0.893617,0.840947,0.810070,0.744444,0.965721,0.845936,0.844983
9,1.000000e-01,1,1,1,trace,3,3330543729,0.149573,0.382979,0.223996,0.236368,0.141026,0.423168,0.226789,0.230390


In [20]:
df_agg = res.groupby(by=['model', 'alpha']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)

In [21]:
df_agg

archi_weight      brainomics_weight      hcp_weight  \
                              mean  std              mean  std       mean   
model    alpha                                                              
factored 1.000000e-07            1  0.0                 1  0.0          1   
         2.682696e-07            1  0.0                 1  0.0          1   
         7.196857e-07            1  0.0                 1  0.0          1   
         1.930698e-06            1  0.0                 1  0.0          1   
         5.179475e-06            1  0.0                 1  0.0          1   
         1.389495e-05            1  0.0                 1  0.0          1   
         3.727594e-05            1  0.0                 1  0.0          1   
         1.000000e-04            1  0.0                 1  0.0          1   
         2.682696e-04            1  0.0                 1  0.0          1   
         7.196857e-04            1  0.0                 1  0.0          1   
         1.930698e-03            1  0.0                 1  0.0          1   
         5.179475e-03            1  0.0                 1  0.0          1   
         1.389495e-02            1  0.0                 1  0.0          1   
         3.727594e-02            1  0.0                 1  0.0          1   
         1.000000e-01            1  0.0                 1  0.0          1   
logistic 1.000000e-07            1  0.0                 1  0.0          1   
         2.682696e-07            1  0.0                 1  0.0          1   
         7.196857e-07            1  0.0                 1  0.0          1   
         1.930698e-06            1  0.0                 1  0.0          1   
         5.179475e-06            1  0.0                 1  0.0          1   
         1.389495e-05            1  0.0                 1  0.0          1   
         3.727594e-05            1  0.0                 1  0.0          1   
         1.000000e-04            1  0.0                 1  0.0          1   
         2.682696e-04            1  0.0                 1  0.0          1   
         7.196857e-04            1  0.0                 1  0.0          1   
         1.930698e-03            1  0.0                 1  0.0          1   
         5.179475e-03            1  0.0                 1  0.0          1   
         1.389495e-02            1  0.0                 1  0.0          1   
         3.727594e-02            1  0.0                 1  0.0          1   
         1.000000e-01            1  0.0                 1  0.0          1   
trace    1.000000e-07            1  0.0                 1  0.0          1   
         2.682696e-07            1  0.0                 1  0.0          1   
         7.196857e-07            1  0.0                 1  0.0          1   
         1.930698e-06            1  0.0                 1  0.0          1   
         5.179475e-06            1  0.0                 1  0.0          1   
         1.389495e-05            1  0.0                 1  0.0          1   
         3.727594e-05            1  0.0                 1  0.0          1   
         1.000000e-04            1  0.0                 1  0.0          1   
         2.682696e-04            1  0.0                 1  0.0          1   
         7.196857e-04            1  0.0                 1  0.0          1   
         1.930698e-03            1  0.0                 1  0.0          1   
         5.179475e-03            1  0.0                 1  0.0          1   
         1.389495e-02            1  0.0                 1  0.0          1   
         3.727594e-02            1  0.0                 1  0.0          1   
         1.000000e-01            1  0.0                 1  0.0          1   

                             rank                  seed                \
                       std   mean       std        mean           std   
model    alpha                                                          
factored 1.000000e-07  0.0  45.75  0.500000  1765347879  1.157716e+09   
         2.682696e-07  0.0  42.75

In [16]:
indices = df_agg.groupby(level='model').aggregate('idxmax')[('test_mean', 'mean')].values
df_agg = df_agg.loc[indices]

In [19]:
df_agg

archi_weight      brainomics_weight      hcp_weight       \
                          mean  std              mean  std       mean  std   
model    alpha                                                               
factored 0.000720            1  0.0                 1  0.0          1  0.0   
logistic 0.000037            1  0.0                 1  0.0          1  0.0   
trace    0.000720            1  0.0                 1  0.0          1  0.0   

                    rank             seed                  ...    test_mean  \
                    mean  std        mean           std    ...         mean   
model    alpha                                             ...                
factored 0.000720  32.75  0.5  1765347879  1.157716e+09    ...     0.908035   
logistic 0.000037  68.00  0.0  1765347879  1.157716e+09    ...     0.904840   
trace    0.000720  46.75  1.5  1765347879  1.157716e+09    ...     0.906209   

                            train_archi           train_brainomics       \
                        std        mean       std             mean  std   
model    alpha                                                            
factored 0.000720  0.005450    0.994231  0.003909              1.0  0.0   
logistic 0.000037  0.003965    0.996581  0.002315              1.0  0.0   
trace    0.000720  0.002955    0.995726  0.003042              1.0  0.0   

                  train_hcp           train_mean            
                       mean       std       mean       std  
model    alpha                                              
factored 0.000720  0.967621  0.000834   0.970825  0.000591  
logistic 0.000037  0.973666  0.000737   0.976353  0.000681  
trace    0.000720  0.968437  0.000645   0.971646  0.000635  

[3 rows x 26 columns]

In [17]:
df_agg[['test_archi', 'test_hcp', 'test_brainomics', 'rank', 'test_mean']]

test_archi            test_hcp           test_brainomics  \
                        mean       std      mean       std            mean   
model    alpha                                                               
factored 0.000720   0.866453  0.018834  0.929692  0.007968        0.919031   
logistic 0.000037   0.863034  0.015167  0.931756  0.009415        0.904846   
trace    0.000720   0.875214  0.014672  0.928729  0.008556        0.900709   

                              rank      test_mean            
                        std   mean  std      mean       std  
model    alpha                                               
factored 0.000720  0.017809  32.75  0.5  0.908035  0.005450  
logistic 0.000037  0.013802  68.00  0.0  0.904840  0.003965  
trace    0.000720  0.014119  46.75  1.5  0.906209  0.002955

Compare results with 3 datasets

In [18]:
transfer_df

test_archi  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0            0.036752   
                                                                 0.5            0.862393   
                                                                 1.0            0.864103   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000003 0    0.0            0.024786   
                                                                 0.5            0.863248   
                                                                 1.0            0.864957   
                                                                 1.5            0.876068   
                                                                 2.0            0.852991   
                                                   0.000010 0    0.0            0.026496   
                                                                 0.5            0.864957   
                                                                 1.0            0.868376   
                                                                 1.5            0.876923   
                                                                 2.0            0.853846   
                                                   0.000032 0    0.0            0.023932   
                                                                 0.5            0.869231   
                                                                 1.0            0.872650   
                                                                 1.5            0.876068   
                                                                 2.0            0.859829   
                                                   0.000100 0    0.0            0.044444   
                                                                 0.5            0.879487   
                                                                 1.0            0.873504   
                                                                 1.5            0.876923   
                                                                 2.0            0.864103   
                                                   0.000316 0    0.0            0.043590   
                                                                 0.5            0.875214   
                                                                 1.0            0.870085   
                                                                 1.5            0.878632   
                                                                 2.0            0.864957   
                                                   0.001000 0    0.0            0.029060   
                                                                 0.5            0.860684   
                                                                 1.0            0.865812   
                                                                 1.5            0.876923   
                                                                 2.0            0.863248   
                                                   0.003162 0    0.0            0.013675   
                                                                 0.5            0.786325   
                                                                 1.0            0.827350   
                                                                 1.5            0.840171   
                                                                 2.0            0.653846   
                                                   0.010000 0    0.0            0.042735   
                              

In [13]:
this_df

hcp_weight  \
                                                                                    mean   
normalized source                 model datasets   alpha    beta archi_weight              
False      hcp_rs_positive_single trace archi__hcp 0.000001 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000003 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000010 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000032 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000100 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.000316 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.001000 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.003162 0    0.0                 2.0   
                                                                 0.5                 1.5   
                                                                 1.0                 1.0   
                                                                 1.5                 0.5   
                                                                 2.0                 0.0   
                                                   0.010000 0    0.0                 2.0   
                              

In [5]:
indices.values

array([(False, 'hcp_rs_positive_single', 'logistic', 'archi', 0.0, 0.0001),
       (False, 'hcp_rs_positive_single', 'trace', 'archi', 0.001, 0.0),
       (False, 'hcp_rs_positive_single', 'trace', 'archi__hcp', 0.00031622776601683794, 0.0)], dtype=object)

In [6]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df[('test_mean', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

In [7]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [8]:
result_df

rank  \
                                                                                                         mean   
target               normalized source                 model    datasets               alpha    beta            
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  17.05   
                                                       trace    brainomics             0.003162 0.0000   9.50   
                                                                brainomics__hcp        0.003162 0.0000  19.05   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  29.05   
                                                       trace    archi                  0.001000 0.0000  18.85   
                                                                archi__hcp             0.000316 0.0000  38.05   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  51.45   

                                                                                                                  \
                                                                                                             std   
target               normalized source                 model    datasets               alpha    beta               
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  0.223607   
                                                       trace    brainomics             0.003162 0.0000  0.512989   
                                                                brainomics__hcp        0.003162 0.0000  0.825578   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  0.223607   
                                                       trace    archi                  0.001000 0.0000  0.670820   
                                                                archi__hcp             0.000316 0.0000  1.050063   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  0.944513   

                                                                                                              seed  \
                                                                                                              mean   
target               normalized source                 model    datasets               alpha    beta                 
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  2330081002   
                                                       trace    brainomics             0.003162 0.0000  2330081002   
                                                                brainomics__hcp        0.003162 0.0000  2330081002   
archi                False      hcp_rs_positive_single logistic archi                  0.000000 0.0001  2330081002   
                                                       trace    archi                  0.001000 0.0000  2330081002   
                                                                archi__hcp             0.000316 0.0000  2330081002   
archi+brainomics+hcp False      hcp_rs_positive_single trace    brainomics__archi__hcp 0.000316 0.0000  2330081002   

                                                                                                                      \
                                                                                                                 std   
target               normalized source                 model    datasets               alpha    beta                   
brainomics           False      hcp_rs_positive_single logistic brainomics             0.000000 0.0000  1.242420e+09   
                                                       trace    brainomics             0.003162 0.0000  1.242420e+09   
                                

In [21]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                   mean   
normalized source                 model datasets               alpha    beta              
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   71.000000   
                                                               0.000001 0.0   71.000000   
                                                               0.000003 0.0   71.000000   
                                                               0.000010 0.0   71.000000   
                                                               0.000032 0.0   71.000000   
                                                               0.000100 0.0   70.666667   
                                                               0.000316 0.0   52.000000   
                                                               0.001000 0.0   28.000000   
                                                               0.003162 0.0   18.333333   
                                                               0.010000 0.0   10.000000   
                                                               0.031623 0.0    3.666667   
                                                               0.100000 0.0    1.000000   
                                                               0.316228 0.0    0.000000   
                                                               1.000000 0.0    0.000000   

                                                                                        \
                                                                                   std   
normalized source                 model datasets               alpha    beta             
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   0.000000   
                                                               0.000001 0.0   0.000000   
                                                               0.000003 0.0   0.000000   
                                                               0.000010 0.0   0.000000   
                                                               0.000032 0.0   0.000000   
                                                               0.000100 0.0   0.577350   
                                                               0.000316 0.0   1.000000   
                                                               0.001000 0.0   0.000000   
                                                               0.003162 0.0   1.154701   
                                                               0.010000 0.0   0.000000   
                                                               0.031623 0.0   0.577350   
                                                               0.100000 0.0   0.000000   
                                                               0.316228 0.0   0.000000   
                                                               1.000000 0.0   0.000000   

                                                                                    seed  \
                                                                                    mean   
normalized source                 model datasets               alpha    beta               
False      hcp_rs_positive_single trace brainomics__archi__hcp 0.000000 0.0   1243615929   
                                                               0.000001 0.0   1243615929   
                                                               0.000003 0.0   1243615929   
                                                               0.000010 0.0   1243615929   
                                                               0.000032 0.0   1243615929   
                                                               0.000100 0.0   1243615929   
                                                               0.000316 0.0   1243615929   
                                                               0.001000 0.0   124361

In [22]:
df

NameError: name 'df' is not defined